<a href="https://colab.research.google.com/github/khanabdulmajid/tensorflow_deep_learning/blob/main/06__Transfer_learning_in_tensorflow_part_3_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#transfer learning with Tensorflow part 3: scaling up

we have seen the power if transfer learning feature extraction and fine tuning , now its time to scale up to all of the classes in food 101(101 toal classes)
Our goal is to beat the original food101 paper with 10% of the training data (leverageing the power of deep learning).

In [1]:
# !wget https://github.com/khanabdulmajid/tensorflow_deep_learning/a90154efbfd5eaf7734bf1e073a94b98d09946dd/helper_functions.py
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-09-03 06:53:01--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-09-03 06:53:01 (57.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]

--2023-09-03 06:53:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.99.128, 142.250.107.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.99.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [2]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [3]:
unzip_data("/content/101_food_classes_10_percent.zip")

In [4]:
!mkdir modelcheckpoint

In [12]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import preprocessing

In [11]:
checkpoint_path="/content/modelcheckpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                       monitor="val_accuracy",
                                                       save_best_only=True,
                                                       save_weights_only=True)
train_dir="/content/101_food_classes_10_percent/train"
test_dir="/content/101_food_classes_10_percent/test"

In [13]:
IMG_SIZE=(224,224)
train_data_10_percent=preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="categorical",
                                                                 image_size=IMG_SIZE
                                                                 )
test_data=preprocessing.image_dataset_from_directory(test_dir,
                                                     label_mode="categorical",
                                                     image_size=IMG_SIZE,
                                                     shuffle=False
                                                     )

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [14]:
data_augmentation=Sequential([
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2)
],  name="data_augmentation")

base_model =tf.keras.applications.EfficientNetB0(include_top=False)

16705208/16705208 [==============================] - 0s 0us/step


In [15]:
len(train_data_10_percent.class_names)

101

In [16]:
base_model.trainable=False

inputs=tf.keras.layers.Input(shape=(224,224,3),name="input_layer")
x = data_augmentation(inputs)
x = base_model(x,training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name="GLobalbalAvgPooling")(x)
outputs=tf.keras.layers.Dense(len(train_data_10_percent.class_names),activation="softmax",name="output_layer_10")(x)
model_0=tf.keras.Model(inputs,outputs)

In [17]:
model_0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, None, None, 3)    0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 GLobalbalAvgPooling (Global  (None, 1280)             0         
 AveragePooling2D)                                               
                                                                 
 output_layer_10 (Dense)     (None, 101)               129381    
                                                                 
Total params: 4,178,952
Trainable params: 129,381
Non-trainab

In [ ]:
model_0.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"]
                )
history_model_0=model_0.fit(train_data_10_percent,
                            epochs=5,
                            validation_data=test_data,
                            validation_steps=int(0.15*len(test_data)),
                            callbacks=[checkpoint_callback]
                )

Epoch 1/5
 20/237 [=>............................] - ETA: 10:56 - loss: 4.6128 - accuracy: 0.0250